In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from scipy.stats import multivariate_normal as mvn
from sklearn.feature_selection import VarianceThreshold

**1-Entrenamiento**

Carga de los datos de entrenamiento 

In [5]:
x_train = pd.read_csv('practica_X_train.csv', sep=';', index_col=0)
y_train = pd.read_csv('practica_Y_train.csv', sep=';', index_col=0)

print(x_train.head())
print(y_train.head())

      Temperature  Humidity  PM2.5  PM10  NO2  SO2   CO  \
41            3.0       6.0    2.0   2.0  4.0  5.0  3.0   
3777          1.0       2.0    0.0   0.0  3.0  1.0  1.0   
2998          2.0       3.0    0.0   0.0  2.0  1.0  1.0   
4469          6.0       2.0    5.0   5.0  5.0  5.0  2.0   
3422          2.0       1.0    0.0   0.0  0.0  2.0  1.0   

      Proximity_to_Industrial_Areas  Population_Density  
41                              0.0                 7.0  
3777                            3.0                 3.0  
2998                            3.0                 0.0  
4469                            0.0                 6.0  
3422                            3.0                 2.0  
     Air_Quality
41          Poor
3777        Good
2998        Good
4469        Poor
3422        Good


1.1 Preprocesado ejemplos

Como el conjunto de datos x_train son datos continuos y en el enunciado se nos dice que no tenemos valores perdidos y en los modelos de Naive Bayes, MVN y GMM los datos de tipo continuo son buenas entradas. 

Por lo tanto, el único preprocesado que sería necesario para poder trabajar con el coinjunto de test es codificar el atributo categórico del conjunto de y_train y aplicar un filtrado por varianza al conjunto de datos de x_train para descartar aquellas columnas que no aporten información. 

In [ ]:
#filtrado por varianza conjunto x_train
thereshold = 0.1

sel = VarianceThreshold(threshold=thereshold)
sel.set_output(transform='pandas')
x_train = sel.fit_transform(x_train)

In [ ]:
#codificación conjunto y_train
categoria = y_train['Air_Quality'].unique()
categoria_codificada ={string:i for i,string in enumerate(categoria)} #diccionario de codificación
cat = y_train['Air_Quality'].map(categoria_codificada)

y_train['Air_Quality'] = cat
print(y_train.head())

      Air_Quality
41              0
3777            1
2998            1
4469            0
3422            1


1.2 Construcción modelo a priori de la etiqueta (cada nivel de calidad del aire tiene un número difernte de ejemplos)

In [7]:
clases = y_train['Air_Quality'].value_counts()  # Cantidad de todas las soluciones distintas
Probs_priori = clases / len(y_train)
print("Probabilidades a priori: ")
print(Probs_priori)

Probabilidades a priori: 
Air_Quality
Good         0.4
Moderate     0.3
Poor         0.2
Hazardous    0.1
Name: count, dtype: float64


1.3 Construir distintos modelos de verosimilitud

Naive Bayes 

In [8]:
naive_bayes = GaussianNB()
naive_bayes.fit(x_train, y_train.values.ravel()) # con y_train.values.ravel() 
                                                 # cambiamos de lista pandas a numpy
                                                 # Es para que solo nos den los valores 
                                                 # Sino te da un enlace raro tambien
print("Probabilidades a priori con Naive Bayes") 
print(naive_bayes.class_prior_) # Solo para comprobar las probabilidades de las clases

predicciones = naive_bayes.predict(x_train)
cont = 0
for y_hat, y_true in zip(predicciones, y_train.values.ravel()):
    if (y_true == y_hat):
        cont += 1
print("acc con conjunto train:", cont / len(predicciones))


Probabilidades a priori con Naive Bayes
[0.4 0.1 0.3 0.2]
acc con conjunto train: 0.9175


MVN

In [9]:
def train_mvn(Y_TRAIN, X_TRAIN):
    estadisticas_modelo = {}
    clases = np.unique(Y_TRAIN)
    for clase in clases:
        datos_clase = X_TRAIN[Y_TRAIN == clase]

        media = np.mean(datos_clase, axis=0)
        covarianza = np.cov(datos_clase, rowvar=False)
        covarianza += np.eye(covarianza.shape[0]) * 1e-6

        modelo = mvn(mean=media, cov=covarianza)

        estadisticas_modelo[clase] = {"media": media, "covarianza": covarianza, "modelo": modelo}
    return estadisticas_modelo

def predict_mvn(x_a_predecir, estadisticas):
    # Predecimos probabilidades primero
    probs = []
    clases = list(estadisticas.keys())
    for clase in clases:
        modelo = estadisticas[clase]["modelo"]
        p = modelo.logpdf(x_a_predecir)
        probs.append(p)
    probs = np.array(probs).T
    y_hat = np.argmax(probs, axis=1)
    return np.array(clases)[y_hat]
    

In [10]:
y_train_flat = y_train.values.ravel()

estadisticas_MVN = train_mvn(y_train_flat, x_train.values)

y_hat = predict_mvn(x_train.values, estadisticas_MVN)
acc = np.mean(y_hat == y_train_flat)
print("acc con conjunto train: ", acc)

acc con conjunto train:  0.92625


GMM

1.4 Construir sistemas con preprocesado, modelo a priori y cada uno de los modelos de verosimilitud

Sistema con Naive Bayes 

Sistema con MVN

Sistema con GMM

1.5 Comparar los sistema con un conjunto de validación  (elegir uno de los sistemas)

1.6 Almacenamiento objetos con Pickle

**2-Implementación e inferencia**

Creación script inferencia

2.1 Cargar objetos guardados del sistema seleccionado

2.2 Cargar conjunto de datos de test

2.3 Ejecución objetos guardados y guardado etiquetas predichas en un CSV con el mismo formato que practica_Y_train.csv

**3-Clustering**

3.1 Realizacion de clastering sobre el conjunto de datos practica_X_train.csv

3.2.1 ¿Cuantos clusters has elegido?¿Elección relacionada con el número de clases? 

3.2.2 Comparacion etiqueta de cada ejemplo de entrenamiento con el cluster asignado ¿Es posible crear una relación entre ambas?

3.2.3 ¿A que cluster se le asignaría cada ejemplo del conjunto de test si se crea una relacion entre cluster y etiqueta? ¿Qué etiqueta asignarias a cada ejemplo de test?

3.2.4 Si tenemos etiquetas asociadas a los datos ¿Se puede utilizar clustering para hacer clasificación?